# A1111 Extension Installer

Installs **Automatic1111** extensions into the WebUI **code tree** so they load properly:

- Code extensions dir: `/opt/stable-diffusion-webui/extensions`
- Data/models root: `/workspace/a1111` (can hold ControlNet/AnimateDiff/InsightFace models, etc.)

**How it works**
- **Defaults**: always installed (safe, lightweight helpers)
- **Recommended**: toggle `enable: True/False` per extension
- **Extra URLs**: paste any Git repo URLs to install as well

**Notes**
- Running again is safe: existing repos are updated with `git pull`.
- After installing, open WebUI → **Settings → Reload UI** (or restart A1111) to load new extensions.

In [ ]:
# --- Paths / settings ---
from pathlib import Path

# A1111 code tree (extensions MUST live here for WebUI to detect them)
CODE_EXT_DIR = Path("/opt/stable-diffusion-webui/extensions")
CODE_EXT_DIR.mkdir(parents=True, exist_ok=True)
print("Extensions directory (CODE):", CODE_EXT_DIR)

# Data dir is still useful for models these extensions may need
DATA_ROOT = Path("/workspace/a1111")  # equals WEBUI_ROOT by default in your image
(DATA_ROOT / "models").mkdir(parents=True, exist_ok=True)
print("Data root (for models):", DATA_ROOT)

# --- Defaults: always install ---
DEFAULT_EXTS = [
    {"name": "Aspect Ratio",       "url": "https://github.com/alemelis/sd-webui-ar"},
    {"name": "Tag Autocomplete",   "url": "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"},
    {"name": "Images Browser",     "url": "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"},
    {"name": "Config Presets",     "url": "https://github.com/Zyin055/Config-Presets"}
]

# --- Recommended: toggle enable True/False ---
RECOMMENDED = [
    {"name": "ControlNet",            "url": "https://github.com/Mikubill/sd-webui-controlnet",                 "enable": false},
    {"name": "ADetailer",             "url": "https://github.com/Bing-su/adetailer",                            "enable": true},
    {"name": "Ultimate SD Upscale",   "url": "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111", "enable": true},
    {"name": "Dynamic Prompts",       "url": "https://github.com/adieyal/sd-dynamic-prompts",                   "enable": false},
    {"name": "Openpose Editor",       "url": "https://github.com/fkunn1326/openpose-editor",                    "enable": false}
]

# --- Extra URLs: paste any additional Git repos here ---
EXTRA_URLS = [
    # e.g. "https://github.com/someuser/some-extension"
]

print("Configured:", len(DEFAULT_EXTS), "defaults,",
      sum(1 for r in RECOMMENDED if r.get("enable")), "recommended enabled,",
      len(EXTRA_URLS), "extra URLs")

In [ ]:
import subprocess, re
from pathlib import Path
from urllib.parse import urlparse

def run(cmd, cwd=None):
    print("$", " ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=True)

def repo_basename(url: str) -> str:
    # turn https://github.com/owner/repo(.git) into "repo"
    path = urlparse(url).path.rstrip("/")
    name = path.split("/")[-1]
    return name[:-4] if name.endswith(".git") else name

def install_or_update(url: str, base: Path) -> Path:
    name = repo_basename(url)
    dst = base / name
    if not dst.exists():
        print(f"\n[install] {name} from {url}")
        run(["git", "clone", "--depth", "1", url, str(dst)])
    else:
        print(f"\n[update] {name}")
        try:
            run(["git", "-C", str(dst), "reset", "--hard"]) 
            run(["git", "-C", str(dst), "pull", "--ff-only"]) 
        except subprocess.CalledProcessError:
            print("  (non-ff; attempting rebase)")
            run(["git", "-C", str(dst), "pull", "--rebase", "--autostash"]) 
    return dst

In [ ]:
installed, errors = [], []

# 1) defaults
for ext in DEFAULT_EXTS:
    try:
        installed.append(install_or_update(ext["url"], CODE_EXT_DIR))
    except Exception as e:
        errors.append((ext["name"], str(e)))

# 2) recommended (enabled only)
for rec in RECOMMENDED:
    if rec.get("enable"):
        try:
            installed.append(install_or_update(rec["url"], CODE_EXT_DIR))
        except Exception as e:
            errors.append((rec["name"], str(e)))

# 3) extras
for url in EXTRA_URLS:
    try:
        installed.append(install_or_update(url, CODE_EXT_DIR))
    except Exception as e:
        errors.append((url, str(e)))

print("\n=== Summary ===")
for p in installed:
    print("✔", p)
if errors:
    print("\nErrors:")
    for name, msg in errors:
        print("✖", name, "→", msg)

## Next steps
- In the WebUI, open **Settings → Reload UI** to load installed extensions.
- For heavy extensions, place models under your data root:

  - ControlNet: `/workspace/a1111/models/ControlNet/*.safetensors|*.pth`
  - AnimateDiff: `/workspace/a1111/models/AnimateDiff/*.safetensors`
  - ReActor (InsightFace):
      - `/workspace/a1111/models/insightface/models/`
      - `/workspace/a1111/models/insightface/antelopev2/`

- If extensions still don’t show, restart the A1111 service:

```bash
supervisorctl -c /etc/supervisor/supervisord.conf restart a1111
```

In [ ]:
# === Optional UI (checkboxes + button) ===
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets==8.*"])  # one-time install
    import ipywidgets as widgets
    from IPython.display import display, clear_output

defaults_info = widgets.HTML("<b>Defaults (always installed):</b> " + ", ".join(d["name"] for d in DEFAULT_EXTS))
reco_checks = [widgets.Checkbox(value=ext.get("enable", False), description=ext["name"]) for ext in RECOMMENDED]
extra_text = widgets.Textarea(
    value="",
    placeholder="One GitHub repo URL per line (e.g. https://github.com/owner/repo)",
    description="Extra URLs:",
    layout=widgets.Layout(width="100%", height="120px")
)
btn = widgets.Button(description="Install / Update", button_style="primary")
out = widgets.Output()

ui = widgets.VBox([
    defaults_info,
    widgets.HTML("<hr><b>Recommended (toggle):</b>"),
    widgets.VBox(reco_checks),
    widgets.HTML("<hr><b>Extra URLs:</b>"),
    extra_text,
    widgets.HTML("<hr>"),
    btn,
    out
])
display(ui)

def on_click(_):
    with out:
        clear_output()
        print("Extensions directory (CODE):", CODE_EXT_DIR)
        urls = [d["url"] for d in DEFAULT_EXTS]
        for chk, ext in zip(reco_checks, RECOMMENDED):
            if chk.value:
                urls.append(ext["url"])
        extras = [line.strip() for line in extra_text.value.splitlines() if line.strip()]
        urls.extend(extras)
        if not urls:
            print("Nothing selected.")
            return
        installed, errors = [], []
        for url in urls:
            try:
                installed.append(install_or_update(url, CODE_EXT_DIR))
            except Exception as e:
                errors.append((url, str(e)))
        print("\n=== Summary ===")
        for p in installed:
            print("✔", p.name)
        if errors:
            print("\nErrors:")
            for url, msg in errors:
                print("✖", url, "→", msg)
        print("\nDone. In WebUI: Settings → Reload UI. If needed, restart A1111 service:")
        print("  supervisorctl -c /etc/supervisor/supervisord.conf restart a1111")

btn.on_click(on_click)